# Dataset

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

In [2]:

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label


def get_dataset():
    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=False,
        as_supervised=True,
        with_info=True,
    )

    ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_train = ds_train.cache()
    # ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    ds_train = ds_train.batch(128)
    ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

    ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds_test = ds_test.batch(128)
    ds_test = ds_test.cache()
    ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

    return ds_train, ds_test


def convert_to_numpy(ds):
    ds_numpy = tfds.as_numpy(ds)
    train_images = None
    train_labels = None
    for ex in ds_numpy:
        if train_images is None:
            train_images = ex[0]
            train_labels = ex[1]
        else:
            train_images = np.concatenate((train_images, ex[0]), axis=0)
            train_labels = np.concatenate((train_labels, ex[1]), axis=0)
    return train_images, train_labels


In [3]:
ds_train, ds_test = get_dataset()
train_images, train_labels = convert_to_numpy(ds_train)
test_images, test_labels = convert_to_numpy(ds_test)

2021-09-17 14:16:35.136036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-17 14:16:35.137931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2021-09-17 14:16:35.137968: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2021-09-17 14:16:35.138002: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-09-17 14:16:35.138034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

## Models

In [4]:
import cvnn.layers as complex_layers

def get_real_mnist_model(dropout=True):
    tf.random.set_seed(0)
    in1 = tf.keras.layers.Input(shape=(28, 28, 1))
    flat = tf.keras.layers.Flatten(input_shape=(28, 28, 1))(in1)
    dense = tf.keras.layers.Dense(128, activation='cart_relu')(flat)
    if dropout:
        dense = tf.keras.layers.Dropout(rate=0.5, seed=116)(dense)
    out = tf.keras.layers.Dense(10, activation='softmax_real_with_abs', kernel_initializer="ComplexGlorotUniform")(dense)
    real_model = tf.keras.Model(in1, out)
    real_model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=['accuracy'],
    )
    return real_model


def get_complex_mnist_model(dropout=True):
    tf.random.set_seed(0)
    inputs = complex_layers.complex_input(shape=(28, 28, 1), dtype=np.float32)
    flat = complex_layers.ComplexFlatten(input_shape=(28, 28, 1), dtype=np.float32)(inputs)
    dense = complex_layers.ComplexDense(128, activation='cart_relu', dtype=np.float32)(flat)
    if dropout:
        dense = complex_layers.ComplexDropout(rate=0.5, seed=116)(dense)
    out = complex_layers.ComplexDense(10, activation='softmax_real_with_abs', dtype=np.float32)(dense)
    complex_model = tf.keras.Model(inputs, out)
    complex_model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=['accuracy'],
    )
    return complex_model

## Using two real models

In [5]:
# Using numpy arrays

complex_model = get_real_mnist_model()
real_model = get_real_mnist_model()

r_history = real_model.fit(train_images, train_labels, epochs=6, validation_data=(test_images, test_labels),
                           verbose=False, shuffle=False)
c_history = complex_model.fit(train_images, train_labels, epochs=6, validation_data=(test_images, test_labels),
                              verbose=False, shuffle=False)
assert r_history.history == c_history.history, f"{r_history.history} != {c_history.history}"

In [6]:
# Using tf dataset

complex_model = get_real_mnist_model()
real_model = get_real_mnist_model()
r_history = real_model.fit(ds_train, epochs=6, validation_data=ds_test,
                           verbose=False, shuffle=False)
c_history = complex_model.fit(ds_train, epochs=6, validation_data=ds_test,
                              verbose=False, shuffle=False)
assert r_history.history == c_history.history, f"{r_history.history} != {c_history.history}"

## Using the cvnn

In [7]:
# Using numpy arrays

complex_model = get_complex_mnist_model()
real_model = get_real_mnist_model()

r_history = real_model.fit(train_images, train_labels, epochs=6, validation_data=(test_images, test_labels),
                           verbose=False, shuffle=False)
c_history = complex_model.fit(train_images, train_labels, epochs=6, validation_data=(test_images, test_labels),
                              verbose=False, shuffle=False)
assert r_history.history == c_history.history, f"{r_history.history} != {c_history.history}"

In [8]:
# Using tf dataset

complex_model = get_complex_mnist_model()
real_model = get_real_mnist_model()
r_history = real_model.fit(ds_train, epochs=6, validation_data=ds_test,
                           verbose=False, shuffle=False)
c_history = complex_model.fit(ds_train, epochs=6, validation_data=ds_test,
                              verbose=False, shuffle=False)
assert r_history.history == c_history.history, f"{r_history.history} != {c_history.history}"

## Without dropout

In [9]:
complex_model = get_complex_mnist_model(dropout=False)
real_model = get_real_mnist_model(dropout=False)
r_history = real_model.fit(ds_train, epochs=6, validation_data=ds_test,
                           verbose=False, shuffle=False)
c_history = complex_model.fit(ds_train, epochs=6, validation_data=ds_test,
                              verbose=False, shuffle=False)
assert r_history.history == c_history.history, f"{r_history.history} != {c_history.history}"